### predictions name - data name
* BCEpredictions - forBCELOSS_4sliced_notSample(x)
2. BCEpredictions_4sliced_notSample - BCELOSS_4sliced_notSample
3. BCEpredictions_4sliced_200000Sample - BCELOSS_4sliced_200000Sample
4. BCEpredictions_4sliced_20000Sample - BCELOSS_4sliced_20000Sample

* BCEpredictions_3sliced - forBCELOSS_3sliced_notSample(x)

In [3]:
sample_num = 20000
sliced_num = 4
epoch_num = 50

In [4]:
data_path = f"/home/aailab/yjh/csv_data/patent_data/edited_data/BCELOSS/{sliced_num}sliced/{sample_num}Sample/"
model_path = f"/home/aailab/yjh/result/multi_label_project/BCELOSS/{sliced_num}sliced/{epoch_num}epoch/"
pretrained_model_path = f"/home/aailab/yjh/result/multi_label_project/BCELOSS/bert_pretrained/{sliced_num}sliced/"

In [5]:
import pandas as pd
import numpy as np
import pickle
import torch

In [6]:
with open(data_path + "labels.pickle", "rb") as rf:
    labels = pickle.load(rf)

len(labels)

637

In [7]:
pred_path = model_path + f"{sample_num}SamplePredictions.pickle"

with open(pred_path, "rb") as rf:
    predictions = pickle.load(rf)

flatten_predictions = np.stack([torch.sigmoid(torch.Tensor(p)) for batch in predictions for p in batch['outputs']])
flatten_predictions.shape

(2000, 637)

In [8]:
pretrained_pred_path = pretrained_model_path + f"{sample_num}SamplePredictions.pickle"

with open(pretrained_pred_path, "rb") as rf:
    pretrained_predictions = pickle.load(rf)

pretrained_flatten_predictions = np.stack([torch.sigmoid(torch.Tensor(p)) for batch in pretrained_predictions for p in batch['outputs']])
pretrained_flatten_predictions.shape

FileNotFoundError: [Errno 2] No such file or directory: '/home/aailab/yjh/result/multi_label_project/BCELOSS/bert_pretrained/4sliced/20000SamplePredictions.pickle'

In [ ]:
import torch
val_path = data_path + 'val_data.csv'
val_data = pd.read_csv(val_path, index_col=0)
val_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 40000 entries, KR1646235B1 to KR1751446B1
Columns: 639 entries, Abstract (Original Language) to H05K
dtypes: int64(637), object(2)
memory usage: 195.3+ MB


In [ ]:
true_label = np.array(val_data[labels])
true_label.shape

(40000, 637)

In [ ]:
binary_predictions = (flatten_predictions >= 0.5).astype(int)
binary_predictions.shape

(40000, 637)

In [ ]:
pretrained_binary_predictions = (pretrained_flatten_predictions >= 0.5).astype(int)
pretrained_binary_predictions.shape

(40000, 637)

In [ ]:
a = np.array([[0, 1, 0, 0, 0], [0, 1, 0, 0, 0], [0, 1, 0, 0, 0]])
b = np.array([[0, 0, 1, 0, 0], [0, 0, 1, 0, 0], [0, 0, 1, 0, 0]])
(a == b).mean()

0.6

In [ ]:
(binary_predictions == true_label).mean()

0.9982372448979592

In [ ]:
(pretrained_binary_predictions == true_label).mean()

0.9982026687598116

In [ ]:
a = np.array([[0, 1, 1, 0, 0], [0, 1, 1, 0, 0]])
b = np.array([[0, 0, 1, 0, 0], [0, 0, 1, 0, 0]])

t_p = (b[a == 1] == 1).sum()
f_p = (b[a == 0] == 1).sum()
t_n = (b[a == 0] == 0).sum()
f_n = (b[a == 1] == 0).sum()
t_p, t_n, f_p, f_n

(2, 6, 0, 2)

# 평가

In [ ]:
whole_data_num = len(true_label) * len(true_label[0])

true_positive = (binary_predictions[true_label == 1] == 1).sum()
true_negative = (binary_predictions[true_label == 0] == 0).sum()
false_positive = (binary_predictions[true_label == 0] == 1).sum()
false_negative = (binary_predictions[true_label == 1] == 0).sum()

In [ ]:
pretrained_true_positive = (pretrained_binary_predictions[true_label == 1] == 1).sum()
pretrained_true_negative = (pretrained_binary_predictions[true_label == 0] == 0).sum()
pretrained_false_positive = (pretrained_binary_predictions[true_label == 0] == 1).sum()
pretrained_false_negative = (pretrained_binary_predictions[true_label == 1] == 0).sum()

In [ ]:
# accuracy
whole_data_num, (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)

(25480000, 0.9982372448979592)

In [ ]:
(pretrained_true_positive + pretrained_true_negative) / (pretrained_true_positive + pretrained_true_negative + pretrained_false_positive + pretrained_false_negative)

0.9982026687598116

In [ ]:
binary_predictions.sum()

63157

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(true_label, binary_predictions), accuracy_score(true_label, pretrained_binary_predictions)

(0.39715, 0.391175)

In [ ]:
# ReCall
recall = true_positive / (true_positive + false_negative)
pretrained_recall = pretrained_true_positive / (pretrained_true_positive + pretrained_false_negative)
recall, pretrained_recall

(0.6330134748877093, 0.6402613311555737)

In [ ]:
precision = true_positive / (true_positive + false_positive)
pretrained_precision = pretrained_true_positive / (pretrained_true_positive + pretrained_false_positive)
precision, pretrained_precision

(0.6872872365691848, 0.6751137901340878)

In [ ]:
f1 = 2 * (precision * recall) / (precision + recall)
pretrained_f1 = 2 * (pretrained_precision * pretrained_recall) / (pretrained_precision + pretrained_recall)
f1, pretrained_f1

(0.6590348366722589, 0.6572258315619293)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
f1 = f1_score(true_label, binary_predictions, average='micro')
precision = precision_score(true_label, binary_predictions, average='micro')
recall = recall_score(true_label, binary_predictions, average='micro')

pretrained_f1 = f1_score(true_label, pretrained_binary_predictions, average='micro')
pretrained_precision = precision_score(true_label, pretrained_binary_predictions, average='micro')
pretrained_recall = recall_score(true_label, pretrained_binary_predictions, average='micro')

(f1, precision, recall), (pretrained_f1, pretrained_precision, pretrained_recall)

((0.6590348366722589, 0.6872872365691848, 0.6330134748877093),
 (0.6572258315619293, 0.6751137901340878, 0.6402613311555737))

In [ ]:
zero_count = (binary_predictions == 0).sum()
one_count = (binary_predictions == 1).sum()
zero_count / one_count

402.4390487198569